# TASK 3

In [1]:
# Apply Decision tree and Random forest and get good accurecy

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("100_Sales.csv")
df.sample(5)

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit,Unnamed: 9,Unnamed: 10
3,Sub_Saharan Africa,Sao Tome and Principe,Fruits,Online,C,07/05/2014,6.92,75591.66,19525.82,NaN,NaN
21,Europe,Portugal,Baby Food,Online,H,09/03/2015,159.42,324971.44,122029.78,NaN,NaN
6,Sub_Saharan Africa,Angola,Household,Offline,M,27/04/2011,502.54,2798046.49,693911.51,NaN,NaN
34,Sub_Saharan Africa,Djibouti,Snacks,Online,M,25/02/2017,97.44,1117953.66,404010.78,NaN,NaN
91,Australia and Oceania,Australia,Beverages,Offline,H,07/11/2014,31.79,445508.05,147031.74,NaN,NaN


In [4]:
# Dropping columns as all values are NaN
df = df.drop(columns=["Unnamed: 9", "Unnamed: 10"])
df.sample(5)

,Region,Country,Item_Type,Sales_Channel,Order_Priority,Ship_Date,Unit_Cost,Total_Revenue,Total_Profit
14,Asia,Mongolia,Personal Care,Offline,C,23/02/2014,56.67,400558.73,122819.06
56,Australia and Oceania,Fiji,Clothes,Offline,C,08/01/2010,35.84,1082418.40,727423.20
51,Sub_Saharan Africa,Lesotho,Fruits,Online,L,18/09/2013,6.92,89623.98,23150.46
28,Europe,Norway,Beverages,Offline,C,30/07/2014,31.79,243133.80,80241.84
20,Europe,Norway,Baby Food,Online,L,28/06/2014,159.42,1901836.00,714157.00


In [5]:
df["Region"].nunique()

7

In [6]:
df["Country"].nunique()

76

In [7]:
df["Item_Type"].nunique()

12

In [8]:
df["Order_Priority"].nunique()

4

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          100 non-null    object 
 1   Country         100 non-null    object 
 2   Item_Type       100 non-null    object 
 3   Sales_Channel   100 non-null    object 
 4   Order_Priority  100 non-null    object 
 5   Ship_Date       100 non-null    object 
 6   Unit_Cost       100 non-null    float64
 7   Total_Revenue   100 non-null    float64
 8   Total_Profit    100 non-null    float64
dtypes: float64(3), object(6)
memory usage: 7.2+ KB


In [10]:
# Removing some unwanted columns:
# Country - As "Regions" give us a geographical information including all countries
# Total_Profit - Can be calculated from "Total_Revenue" column.
# Ship_Date - Not important but can use it by transforming to date/year etc.
df1 = df.drop(columns=["Country", "Ship_Date", "Total_Profit"])
df1.sample(5)

,Region,Item_Type,Sales_Channel,Order_Priority,Unit_Cost,Total_Revenue
85,North America,Personal Care,Offline,L,56.67,524870.06
3,Sub_Saharan Africa,Fruits,Online,C,6.92,75591.66
56,Australia and Oceania,Clothes,Offline,C,35.84,1082418.40
55,Sub_Saharan Africa,Clothes,Online,C,35.84,380512.96
72,Sub_Saharan Africa,Beverages,Online,C,31.79,272410.45


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          100 non-null    object 
 1   Item_Type       100 non-null    object 
 2   Sales_Channel   100 non-null    object 
 3   Order_Priority  100 non-null    object 
 4   Unit_Cost       100 non-null    float64
 5   Total_Revenue   100 non-null    float64
dtypes: float64(2), object(4)
memory usage: 4.8+ KB


In [12]:
# Tranforming all categorical columns to numerical coumns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['Region'] = le.fit_transform(df1['Region'])
df1['Item_Type'] = le.fit_transform(df1['Item_Type'])
df1['Sales_Channel'] = le.fit_transform(df1['Sales_Channel'])
df1['Order_Priority'] = le.fit_transform(df1['Order_Priority'])
df1.sample(5)

,Region,Item_Type,Sales_Channel,Order_Priority,Unit_Cost,Total_Revenue
78,3,0,0,1,159.42,2198981.92
21,3,0,1,1,159.42,324971.44
61,3,0,1,2,159.42,1212580.00
44,0,3,1,1,35.84,648030.40
10,0,11,1,1,90.93,19103.44


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          100 non-null    int32  
 1   Item_Type       100 non-null    int32  
 2   Sales_Channel   100 non-null    int32  
 3   Order_Priority  100 non-null    int32  
 4   Unit_Cost       100 non-null    float64
 5   Total_Revenue   100 non-null    float64
dtypes: float64(2), int32(4)
memory usage: 3.3 KB


# Splitting of Data to Train and Test

In [14]:
# Splittind the data : 20% Test and 80% Train
from sklearn.model_selection import train_test_split
x = df1.drop('Total_Revenue', axis=1)
y = df1['Total_Revenue']               # Target column
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestRegressor
randomForest_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
randomForest_model.fit(x_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [16]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred_randomForest = randomForest_model.predict(x_test)

r2_score_randomForest = r2_score(y_test, y_pred_randomForest)
mse_score_randomForest = mean_squared_error(y_test, y_pred_randomForest)

print("R2 Score:", r2_score_randomForest)
print("MSE:", mse_score_randomForest)

R2 Score: 0.6493700512881926
MSE: 776216273543.3202


In [27]:
# Using GridSearch to improve accuracy
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_randomForest = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='r2')
grid_search_randomForest.fit(x_train, y_train)
print("Best Parameters:", grid_search_randomForest.best_params_)

best_randomForest_model = grid_search_randomForest.best_estimator_
best_r2_score_randomForest = best_randomForest_model.score(x_test, y_test)
print(f"R2 Score with GridSearch: {best_r2_score_randomForest}")

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
R2 Score with GridSearch: 0.650163868004193


In [28]:
print("R2 Score:", r2_score_randomForest)
print(f"R2 Score with GridSearch: {best_r2_score_randomForest} \n")

if r2_score_randomForest > best_r2_score_randomForest:
    print("Best R2 Score: ",r2_score_randomForest)
else:
    print("Best R2 Score: ",best_r2_score_randomForest)

R2 Score: 0.6493700512881926
R2 Score with GridSearch: 0.650163868004193 

Best R2 Score:  0.650163868004193


# Decision Tree Classifier

In [20]:
from sklearn.tree import DecisionTreeRegressor
decisionTree_model = DecisionTreeRegressor(max_depth=10, random_state=42)
decisionTree_model.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=42)

In [21]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred_decisionTree = decisionTree_model.predict(x_test)

r2_score_decisionTree = r2_score(y_test, y_pred_decisionTree)
mse_score_decisionTree = mean_squared_error(y_test, y_pred_decisionTree)

print("R2 Score:", r2_score_decisionTree)
print("MSE:", mse_score_decisionTree)

R2 Score: 0.3473901772711624
MSE: 1444732164315.7935


In [22]:
# Using GridSearch to improve accuracy
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_decisionTree = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5, scoring='r2')
grid_search_decisionTree.fit(x_train, y_train)
print("Best Parameters:", grid_search_decisionTree.best_params_)

best_decisionTree_model = grid_search_decisionTree.best_estimator_
best_r2_score_decisionTree = best_decisionTree_model.score(x_test, y_test)
print(f"R2 Score with GridSearch: {best_r2_score_decisionTree}")

Best Parameters: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
R2 Score with GridSearch: 0.5870064492992488


In [23]:
print("R2 Score:", r2_score_decisionTree)
print(f"R2 Score with GridSearch: {best_r2_score_decisionTree} \n")

if r2_score_decisionTree > best_r2_score_decisionTree:
    print("Best R2 Score: ",r2_score_decisionTree)
else:
    print("Best R2 Score: ",best_r2_score_decisionTree)

R2 Score: 0.3473901772711624
R2 Score with GridSearch: 0.5870064492992488 

Best R2 Score:  0.5870064492992488
